In [39]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [24]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text


# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')


# create a list to store neighborhood data
neighborhoodList = []


# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [25]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [26]:
kl_df.shape

(71, 1)

In [27]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [28]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [29]:
coords

[[3.0576900000000364, 101.74388000000005],
 [3.1531525006886456, 101.70041313127312],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072620000000029, 101.71471000000008],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1347800000000348, 101.67262000000005],
 [3.111020000000053, 101.66283000000004],
 [3.09898000000004, 101.73499000000004],
 [3.1357600000000616, 101.70837000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.057800000000043, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.152017197420035, 101.70102760046613],
 [3.129290000000026, 101.69892000000004],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.147980000000075, 101.66798000000006],
 [3.1387586696676304, 101.6840455304707],
 [3.1387586696676304, 101.6840455304707],
 [3.156685250705136, 101.69807

In [30]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [31]:

# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [32]:
print(kl_df.shape)
kl_df

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.153153,101.700413
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072620,101.714710
5,Bandar Tun Razak,3.082800,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.134780,101.672620
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.098980,101.734990


In [33]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

In [34]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516636, 101.6943028.


In [35]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [36]:
map_kl.save('map_kl.html')

In [37]:
# define Foursquare Credentials and Version
CLIENT_ID = '30RUQWZORB3T3FKLWJOUEAXQ10ICIGS1TKTZWKS22CIAWK0H' # your Foursquare ID
CLIENT_SECRET = 'E2ICY0VKBWIYVSSWQGIYYGSIS2PFWNBNA0YO5ZIFCGMEEMY4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 30RUQWZORB3T3FKLWJOUEAXQ10ICIGS1TKTZWKS22CIAWK0H
CLIENT_SECRET:E2ICY0VKBWIYVSSWQGIYYGSIS2PFWNBNA0YO5ZIFCGMEEMY4


In [40]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [41]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7086, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
2,Alam Damai,3.05769,101.74388,Restoran Ikbal,3.061134,101.750220,Restaurant
3,Alam Damai,3.05769,101.74388,閒茶素食店 Leisure Tea Vegetarian,3.057673,101.747258,Vegetarian / Vegan Restaurant
4,Alam Damai,3.05769,101.74388,Ivy Sekinchan Seafood Noodle House 适耕莊特制魚丸海鲜面,3.065749,101.748718,Noodle House


In [42]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,99,99,99,99,99,99
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


In [43]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 310 uniques categories.


In [44]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supplement Shop', 'Noodle House', 'Restaurant',
       'Vegetarian / Vegan Restaurant', 'Chinese Restaurant',
       'Breakfast Spot', 'Food Court', 'Asian Restaurant',
       'Other Great Outdoors', 'Park', 'Snack Place',
       'Dim Sum Restaurant', 'Food Truck', 'Japanese Restaurant',
       'Indian Restaurant', 'Bubble Tea Shop', 'Spa',
       'Seafood Restaurant', 'Chinese Breakfast Place',
       'Convenience Store', 'Dessert Shop', 'Cantonese Restaurant',
       'Farmers Market', 'Bakery', 'Pet Store', 'Café',
       'Malay Restaurant', 'Outlet Store', 'Gym / Fitness Center',
       'Hakka Restaurant', 'Steakhouse', 'Badminton Court',
       'Fast Food Restaurant', 'Athletics & Sports',
       'Middle Eastern Restaurant', 'Burger Joint', 'Winery',
       'Padangnese Restaurant', 'College Bookstore', 'Grocery Store',
       'Botanical Garden', 'Halal Restaurant', 'Shoe Store',
       'Coffee Shop', 'Monument / Landmark', 'Hostel',
       'Latin American Restaurant', 'Hote

In [45]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [46]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7086, 311)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Padangnese Restaurant,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex

In [47]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(71, 311)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Padangnese Restaurant,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex

In [48]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

39

In [49]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [50]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.000000
1,"Ampang, Kuala Lumpur",0.030000
2,Bandar Menjalara,0.010000
3,Bandar Sri Permaisuri,0.000000
4,Bandar Tasik Selatan,0.010101


In [51]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 2, 1, 2, 1, 0, 0, 2, 1], dtype=int32)

In [52]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [53]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.000000,1
1,"Ampang, Kuala Lumpur",0.030000,0
2,Bandar Menjalara,0.010000,2
3,Bandar Sri Permaisuri,0.000000,1
4,Bandar Tasik Selatan,0.010101,2


In [54]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.000000,1,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.030000,0,3.153153,101.700413
2,Bandar Menjalara,0.010000,2,3.190350,101.625450
3,Bandar Sri Permaisuri,0.000000,1,3.103910,101.712260
4,Bandar Tasik Selatan,0.010101,2,3.072620,101.714710


In [55]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
40,Mont Kiara,0.030000,0,3.165320,101.652430
36,Lembah Pantai,0.040000,0,3.121202,101.663899
15,Bukit Nanas,0.030000,0,3.152017,101.701028
42,"Pudu, Kuala Lumpur",0.040000,0,3.133540,101.713070
11,Brickfields,0.030000,0,3.129160,101.684060
7,Bangsar Park,0.050000,0,3.134780,101.672620
51,Sri Hartamas,0.030000,0,3.162200,101.650360
66,Taman Tun Dr Ismail,0.030000,0,3.152830,101.622710
67,Taman U-Thant,0.040000,0,3.157700,101.724520
30,KL Eco City,0.030000,0,3.117140,101.673890


In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [58]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
40,Mont Kiara,0.03,0,3.165320,101.652430
36,Lembah Pantai,0.04,0,3.121202,101.663899
15,Bukit Nanas,0.03,0,3.152017,101.701028
42,"Pudu, Kuala Lumpur",0.04,0,3.133540,101.713070
11,Brickfields,0.03,0,3.129160,101.684060
7,Bangsar Park,0.05,0,3.134780,101.672620
51,Sri Hartamas,0.03,0,3.162200,101.650360
66,Taman Tun Dr Ismail,0.03,0,3.152830,101.622710
67,Taman U-Thant,0.04,0,3.157700,101.724520
30,KL Eco City,0.03,0,3.117140,101.673890


In [59]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
37,Maluri,0.0,1,3.147890,101.694050
39,Miharja,0.0,1,3.147890,101.694050
48,Setapak,0.0,1,3.188160,101.704150
41,Pantai Dalam,0.0,1,3.094760,101.667470
44,Salak South,0.0,1,3.081020,101.697240
46,Semarak,0.0,1,3.179927,101.721442
47,Sentul Raya,0.0,1,3.187431,101.691453
0,Alam Damai,0.0,1,3.057690,101.743880
56,Taman Connaught,0.0,1,3.082690,101.736890
54,Taman Bukit Maluri,0.0,1,3.200660,101.633370


In [60]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,Bangsar South,0.020000,2,3.111020,101.662830
4,Bandar Tasik Selatan,0.010101,2,3.072620,101.714710
10,"Batu, Kuala Lumpur",0.020000,2,3.135760,101.708370
12,Bukit Bintang,0.020000,2,3.147770,101.708550
2,Bandar Menjalara,0.010000,2,3.190350,101.625450
57,Taman Desa,0.010000,2,3.102970,101.684710
16,Bukit Petaling,0.010000,2,3.129290,101.698920
17,Bukit Tunku,0.020000,2,3.173810,101.682760
64,Taman Sri Sinar,0.010101,2,3.190070,101.652930
32,Kampung Datuk Keramat,0.010000,2,3.166400,101.730460
